# Risky Biscuts Final Project

Group Members: <br>
Alexandria LaBatte, <br>
Ernest Elizondo, <br>
Rebecca Garcia <br>

## Load Dataset

In [1]:
import csv

import numpy as np
np.random.seed(42)
import random
random.seed(42)

## Load Datasets

X_txt_train = []
y_train = []
X_txt_test = []
y_test = []

# Load train dataset
with open("train.tsv", encoding = 'UTF-8') as fileTrain:
    readerTrain = csv.reader(fileTrain, delimiter='\t', quoting = csv.QUOTE_NONE)
    for row in readerTrain:
        X_txt_train.append(row[1])
        y_train.append(row[2])

fileTrain.close()

# Load test dataset
with open("test.tsv", encoding = 'UTF-8') as fileTest:
    readerTest = csv.reader(fileTest, delimiter='\t', quoting = csv.QUOTE_NONE)
    for row in readerTest:
        X_txt_test.append(row[1])
        y_test.append(row[2])

fileTest.close()

In [2]:
num_tweets = 0
for text, y  in zip(X_txt_train, y_train):
    print('#', num_tweets + 1, "Tweet: {}".format(text))
    #print("Tweet: {}".format(text))
    print("Ground-Truth Class: {}".format(y))
    
    num_tweets += 1
    if num_tweets == 20:
        break

# 1 Tweet: @USER She should ask a few native Americans what their take on this is.
Ground-Truth Class: UNT
# 2 Tweet: @USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL
Ground-Truth Class: TIN
# 3 Tweet: Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT
Ground-Truth Class: NOT
# 4 Tweet: @USER Someone should'veTaken" this piece of shit to a volcano. 😂"
Ground-Truth Class: UNT
# 5 Tweet: @USER @USER Obama wanted liberals &amp; illegals to move into red states
Ground-Truth Class: NOT
# 6 Tweet: @USER Liberals are all Kookoo !!!
Ground-Truth Class: TIN
# 7 Tweet: @USER @USER Oh noes! Tough shit.
Ground-Truth Class: UNT
# 8 Tweet: @USER was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism
Ground-Truth Class: TIN
# 9 Tweet: @US

## Features

#### Feature set 1

In [3]:
import re

# regex to match '.', '!', '?'
def cnt(x):
    result = re.findall(r'(\.{3}|\!{3}|\?{3})', x)
    if len(result) > 0:
        return 1
    else:
        return 0

In [4]:
# testing RegEx
#myString = "omg!!!"
#cnt(myString)

In [5]:
# for X_txt_train
X_train_exc_features = []
for z in X_txt_train:
    X_train_exc_features.append([cnt(z)])
#X_train_exc_features


# for X_txt_test
X_test_exc_features = []
for n in X_txt_test:
    X_test_exc_features.append([cnt(n)])
#X_test_exc_features

In [6]:
# Testing it here 
import numpy as np

exc_matrix = np.array(X_train_exc_features)
exc_matrix
#exc_matrix.shape

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

#### Feature set 2

In [7]:
import numpy as np

pronouns = ['you','your','yours','he','his','she','it','they','her',
            'hers','them','theirs',"you\'re","she\'s","he\'s",'these']

def targeted(string):
    tgt=0
    for word in string.lower().split():
            if word in pronouns:
                tgt+=1 # would it be tgt += 1?
    return tgt

X_train_tgt_features = []
for z in X_txt_train:
    X_train_tgt_features.append([targeted(z)])

X_test_tgt_features = []
for n in X_txt_test:
    X_test_tgt_features.append([targeted(n)])
    
tgt_matrix = np.array(X_train_tgt_features)
tgt_matrix

array([[1],
       [0],
       [0],
       ...,
       [0],
       [4],
       [1]])

#### Feature set 3

In [8]:
# word classifier class
class WordClassifier():
    def __init__(self):
        self.badwords = set()
        with open('badwords.txt') as file:
            for row in file:
                self.badwords.add(row.strip())

                
    def bad(self, string):
        bdwd=0
        for word in string.lower().split():
            if word in self.badwords:
                bdwd+=1 # bdwd += 1 ?
        return bdwd
    
wordClass = WordClassifier()

X_train_bdwdd_features = []
for z in X_txt_train:
    X_train_bdwdd_features.append([wordClass.bad(z)])

X_test_bdwdd_features = []
for n in X_txt_test:
    X_test_bdwdd_features.append([wordClass.bad(n)])
    
bdwd_matrix = np.array(X_train_bdwdd_features)
bdwd_matrix

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

## Important Features

In [9]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

X_matrix = vec.fit_transform(X_txt_train)

print(X_matrix.shape)

skb = SelectKBest(chi2, k=20).fit(X_matrix, y_train)

feature_imp = np.array(vec.get_feature_names())

print(feature_imp[skb.get_support(indices=True)])

(10592, 16829)
['ass' 'bitch' 'bullshit' 'console' 'crap' 'crazy' 'disgusting' 'fuck'
 'fuckin' 'fucking' 'he' 'idiot' 'liar' 'racist' 'sex' 'shit' 'stupid'
 'the' 'user' 'your']


## Draft model w/ no features

In [10]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

vec = CountVectorizer(ngram_range=(1, 1))

X_train = vec.fit_transform(X_txt_train) 
X_test = vec.transform(X_txt_test) 

linSVC = LinearSVC()

params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

clf = GridSearchCV(linSVC, params, scoring ='f1_macro', cv=5)

clf.fit(X_train, y_train)

validation_score = clf.best_score_ 
print("Validation F1: {:.4f}".format(validation_score))

svm_test_predictions = clf.predict(X_test)  

precision = precision_score(svm_test_predictions, y_test, average = "macro") 
recall = recall_score(svm_test_predictions, y_test, average = "macro")
f1 = f1_score(svm_test_predictions, y_test, average = "macro")

print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))

C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

Validation F1: 0.4765
Precision: 0.2379
Recall: 0.3333
F1: 0.2777


C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LinearSVC with feature 1

In [11]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import scipy.sparse as sp
from scipy.sparse import hstack

import numpy as np
np.random.seed(42)
import random
random.seed(42)

vec = CountVectorizer(ngram_range=(1, 1))

X_train_exxc = vec.fit_transform(X_txt_train)
X_test_exxc = vec.transform(X_txt_test)

arrayTrain = np.array(X_train_exc_features)
arrayTest = np.array(X_test_exc_features)

X_train_exc = hstack([X_train_exxc, arrayTrain])
X_test_exc = hstack([X_test_exxc, arrayTest])

linSVC = LinearSVC(random_state=42)

params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

# I added two gridsearchcv and fits so we can get both micro and macro

clf1 = GridSearchCV(linSVC, params, scoring ='f1_macro', cv=5)
clf2 = GridSearchCV(linSVC, params, scoring ='f1_micro', cv=5)

clf1.fit(X_train_exc, y_train)
clf2.fit(X_train_exc, y_train)

C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

GridSearchCV(cv=5, estimator=LinearSVC(random_state=42),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]},
             scoring='f1_micro')

### Evaluate LinearSVC Feature 1

In [12]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf1.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf2.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

svm_test_predictions1 = clf1.predict(X_test_exc)
svm_test_predictions2 = clf2.predict(X_test_exc)

precision1 = precision_score(svm_test_predictions1, y_test, average = "macro") 
recall1 = recall_score(svm_test_predictions1, y_test, average = "macro")
f1_1 = f1_score(svm_test_predictions1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(svm_test_predictions2, y_test, average = "micro") 
recall2 = recall_score(svm_test_predictions2, y_test, average = "micro")
f1_2 = f1_score(svm_test_predictions2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4757
Validation F1_micro: 0.7315

F1 Macro
Precision: 0.2378
Recall: 0.3333
F1: 0.2776

F1 Micro
Precision: 0.7776
Recall: 0.7776
F1: 0.7776


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LinearSVC with feature 2

In [13]:
import scipy.sparse as sp
from scipy.sparse import hstack

vec = CountVectorizer(ngram_range=(1, 1))

X_train_tggt = vec.fit_transform(X_txt_train)
X_test_tggt = vec.transform(X_txt_test)

newarrayTrain = np.array(X_train_tgt_features)
newarrayTest = np.array(X_test_tgt_features)

X_train_tgt = hstack([X_train_tggt, newarrayTrain])
X_test_tgt = hstack([X_test_tggt, newarrayTest])

linSVC = LinearSVC(random_state=42)

params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

# I added two gridsearchcv and fits so we can get both micro and macro

clf_tgt1 = GridSearchCV(linSVC, params, scoring ='f1_macro', cv=5)
clf_tgt2 = GridSearchCV(linSVC, params, scoring ='f1_micro', cv=5)

clf_tgt1.fit(X_train_tgt, y_train)
clf_tgt2.fit(X_train_tgt, y_train)

C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

GridSearchCV(cv=5, estimator=LinearSVC(random_state=42),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]},
             scoring='f1_micro')

### Evaluate LinearSVC Feature 2

In [14]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_tgt1.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_tgt2.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

svm_test_predictions1 = clf_tgt1.predict(X_test_tgt)
svm_test_predictions2 = clf_tgt2.predict(X_test_tgt)

#print(classification_report(y_test, svm_test_predictions))

precision1 = precision_score(svm_test_predictions1, y_test, average = "macro") 
recall1 = recall_score(svm_test_predictions1, y_test, average = "macro")
f1_1 = f1_score(svm_test_predictions1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(svm_test_predictions2, y_test, average = "micro") 
recall2 = recall_score(svm_test_predictions2, y_test, average = "micro")
f1_2 = f1_score(svm_test_predictions2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4769
Validation F1_micro: 0.7304

F1 Macro
Precision: 0.2378
Recall: 0.3333
F1: 0.2776

F1 Micro
Precision: 0.7753
Recall: 0.7753
F1: 0.7753


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LinearSVC with feature 3

In [15]:
import scipy.sparse as sp
from scipy.sparse import hstack

vec = CountVectorizer(ngram_range=(1, 1))

X_train_bdwdd = vec.fit_transform(X_txt_train)
X_test_bdwdd = vec.transform(X_txt_test)

arrTrain = np.array(X_train_bdwdd_features)
arrTest = np.array(X_test_bdwdd_features)

X_train_bdwd = hstack([X_train_bdwdd, arrTrain])
X_test_bdwd = hstack([X_test_bdwdd, arrTest])

linSVC = LinearSVC(random_state=42)

params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

# I added two gridsearchcv and fits so we can get both micro and macro

clf_bdwd1 = GridSearchCV(linSVC, params, scoring ='f1_macro', cv=5)
clf_bdwd2 = GridSearchCV(linSVC, params, scoring ='f1_micro', cv=5)

clf_bdwd1.fit(X_train_bdwd, y_train)
clf_bdwd2.fit(X_train_bdwd, y_train)

C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\astar\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

GridSearchCV(cv=5, estimator=LinearSVC(random_state=42),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]},
             scoring='f1_micro')

### Evaluate LinearSVC Feature 3

In [16]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_bdwd1.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_bdwd2.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

svm_test_predictions1 = clf_bdwd1.predict(X_test_bdwd)
svm_test_predictions2 = clf_bdwd2.predict(X_test_bdwd)

#print(classification_report(y_test, svm_test_predictions))

precision1 = precision_score(svm_test_predictions1, y_test, average = "macro") 
recall1 = recall_score(svm_test_predictions1, y_test, average = "macro")
f1_1 = f1_score(svm_test_predictions1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(svm_test_predictions2, y_test, average = "micro") 
recall2 = recall_score(svm_test_predictions2, y_test, average = "micro")
f1_2 = f1_score(svm_test_predictions2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4809
Validation F1_micro: 0.7316

F1 Macro
Precision: 0.2379
Recall: 0.3333
F1: 0.2777

F1 Micro
Precision: 0.7753
Recall: 0.7753
F1: 0.7753


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RandomForest with feature 1

In [17]:
from sklearn.ensemble import RandomForestClassifier
np.random.seed(42)
random.seed(42)

X_train_exxc = vec.fit_transform(X_txt_train)
X_test_exxc = vec.transform(X_txt_test)

arrayTrain = np.array(X_train_exc_features)
arrayTest = np.array(X_test_exc_features)

X_train_exc = hstack([X_train_exxc, arrayTrain])
X_test_exc = hstack([X_test_exxc, arrayTest])

params = {'n_estimators': [10, 50, 100]}

rf = RandomForestClassifier(random_state=42)

# I added two gridsearchcv and fits so we can get both micro and macro

clf_rf1exc = GridSearchCV(rf, params, scoring = 'f1_macro', cv=2)
clf_rf2exc = GridSearchCV(rf, params, scoring = 'f1_micro', cv=2)

clf_rf1exc.fit(X_train_exc, y_train)
clf_rf2exc.fit(X_train_exc, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42),
             param_grid={'n_estimators': [10, 50, 100]}, scoring='f1_micro')

### Evaluate RandomForest Feature 1

In [19]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_rf1exc.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_rf2exc.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

preds1 = clf_rf1exc.predict(X_test_exc)
preds2 = clf_rf2exc.predict(X_test_exc)

precision1 = precision_score(preds1, y_test, average = "macro") 
recall1 = recall_score(preds1, y_test, average = "macro")
f1_1 = f1_score(preds1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(preds2, y_test, average = "micro") 
recall2 = recall_score(preds2, y_test, average = "micro")
f1_2 = f1_score(preds2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4689
Validation F1_micro: 0.7338

F1 Macro
Precision: 0.2871
Recall: 0.3333
F1: 0.3085

F1 Micro
Precision: 0.8614
Recall: 0.8614
F1: 0.8614


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RandomForest with feature 2

In [20]:
from sklearn.ensemble import RandomForestClassifier
# recommend n_estimators and criterion
np.random.seed(42)
random.seed(42)

X_train_tggt = vec.fit_transform(X_txt_train)
X_test_tggt = vec.transform(X_txt_test)

newarrayTrain = np.array(X_train_tgt_features)
newarrayTest = np.array(X_test_tgt_features)

X_train_tgt = hstack([X_train_tggt, newarrayTrain])
X_test_tgt = hstack([X_test_tggt, newarrayTest])

params = {'n_estimators': [10, 50, 100]}

rf = RandomForestClassifier(random_state=42)

# I added two gridsearchcv and fits so we can get both micro and macro

clf_rf1tgt = GridSearchCV(rf, params, scoring = 'f1_macro', cv=2)
clf_rf2tgt = GridSearchCV(rf, params, scoring = 'f1_micro', cv=2)

clf_rf1tgt.fit(X_train_tgt, y_train)
clf_rf2tgt.fit(X_train_tgt, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42),
             param_grid={'n_estimators': [10, 50, 100]}, scoring='f1_micro')

### Evaluate RandomForest Feature 2

In [22]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_rf1tgt.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_rf2tgt.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

preds1 = clf_rf1tgt.predict(X_test_tgt)
preds2 = clf_rf2tgt.predict(X_test_tgt)

precision1 = precision_score(preds1, y_test, average = "macro") 
recall1 = recall_score(preds1, y_test, average = "macro")
f1_1 = f1_score(preds1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(preds2, y_test, average = "micro") 
recall2 = recall_score(preds2, y_test, average = "micro")
f1_2 = f1_score(preds2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4689
Validation F1_micro: 0.7320

F1 Macro
Precision: 0.2902
Recall: 0.3333
F1: 0.3102

F1 Micro
Precision: 0.8705
Recall: 0.8705
F1: 0.8705


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RandomForest with feature 3

In [57]:
from sklearn.ensemble import RandomForestClassifier
# recommend n_estimators and criterion
np.random.seed(42)
random.seed(42)

X_train_bdwdd = vec.fit_transform(X_txt_train)
X_test_bdwdd = vec.transform(X_txt_test)

arrTrain = np.array(X_train_bdwdd_features)
arrTest = np.array(X_test_bdwdd_features)

X_train_bdwd = hstack([X_train_bdwdd, arrTrain])
X_test_bdwd = hstack([X_test_bdwdd, arrTest])

params = {'n_estimators': [10, 50, 100]}

rf = RandomForestClassifier(random_state=42)

# I added two gridsearchcv and fits so we can get both micro and macro

clf_rf1bdwd = GridSearchCV(rf, params, scoring = 'f1_macro', cv=2)
clf_rf2bdwd = GridSearchCV(rf, params, scoring = 'f1_micro', cv=2)

clf_rf1bdwd.fit(X_train_bdwd, y_train)
clf_rf2bdwd.fit(X_train_bdwd, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42),
             param_grid={'n_estimators': [10, 50, 100]}, scoring='f1_micro')

### Evaluate RandomForest Feature 3

In [58]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_rf1bdwd.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_rf2bdwd.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

preds1 = clf_rf1bdwd.predict(X_test_bdwd)
preds2 = clf_rf2bdwd.predict(X_test_bdwd)

precision1 = precision_score(preds1, y_test, average = "macro") 
recall1 = recall_score(preds1, y_test, average = "macro")
f1_1 = f1_score(preds1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(preds2, y_test, average = "micro") 
recall2 = recall_score(preds2, y_test, average = "micro")
f1_2 = f1_score(preds2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4739
Validation F1_micro: 0.7385

F1 Macro
Precision: 0.2807
Recall: 0.3333
F1: 0.3048

F1 Micro
Precision: 0.8418
Recall: 0.8418
F1: 0.8418


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
import csv

with open('test.tsv',"r", encoding="UTF-8") as source:
    reader= csv.reader(source, delimiter='\t')
    with open('Risky_Biscuits_test_pred.tsv', 'w', encoding="UTF-8") as result:
        writer=csv.writer(result, delimiter='\t')
        for row,i in zip(reader,preds2):
            row[2]=i
            #print(row)
            writer.writerows([row])

## DecisionTree with feature 1

In [40]:
from sklearn.tree import DecisionTreeClassifier
np.random.seed(42)
random.seed(42)

X_train_exxc = vec.fit_transform(X_txt_train)
X_test_exxc = vec.transform(X_txt_test)

arrayTrain = np.array(X_train_exc_features)
arrayTest = np.array(X_test_exc_features)

X_train_exc = hstack([X_train_exxc, arrayTrain])
X_test_exc = hstack([X_test_exxc, arrayTest])

params = {'min_samples_split': [2, 3, 4]}

dt = DecisionTreeClassifier(random_state=42)

# I added two gridsearchcv and fits so we can get both micro and macro

clf_dt1 = GridSearchCV(dt, params, scoring = 'f1_macro', cv=2)
clf_dt2 = GridSearchCV(dt, params, scoring = 'f1_micro', cv=2)

clf_dt1.fit(X_train_exc, y_train)
clf_dt2.fit(X_train_exc, y_train)

GridSearchCV(cv=2, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_samples_split': [2, 3, 4]}, scoring='f1_micro')

### Evaluate DecisionTree Feature 1

In [41]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_dt1.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_dt2.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

preds1 = clf_dt1.predict(X_test_exc)
preds2 = clf_dt2.predict(X_test_exc)

precision1 = precision_score(preds1, y_test, average = "macro") 
recall1 = recall_score(preds1, y_test, average = "macro")
f1_1 = f1_score(preds1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(preds2, y_test, average = "micro") 
recall2 = recall_score(preds2, y_test, average = "micro")
f1_2 = f1_score(preds2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4629
Validation F1_micro: 0.6848

F1 Macro
Precision: 0.2407
Recall: 0.3333
F1: 0.2795

F1 Micro
Precision: 0.7221
Recall: 0.7221
F1: 0.7221


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## DecisionTree with feature 2

In [46]:
from sklearn.tree import DecisionTreeClassifier
np.random.seed(42)
random.seed(42)

X_train_tggt = vec.fit_transform(X_txt_train)
X_test_tggt = vec.transform(X_txt_test)

newarrayTrain = np.array(X_train_tgt_features)
newarrayTest = np.array(X_test_tgt_features)

X_train_tgt = hstack([X_train_tggt, newarrayTrain])
X_test_tgt = hstack([X_test_tggt, newarrayTest])

params = {'min_samples_split': [2, 3, 4]}

dt = DecisionTreeClassifier(random_state=42)

# I added two gridsearchcv and fits so we can get both micro and macro

clf_dt1tgt = GridSearchCV(dt, params, scoring = 'f1_macro', cv=2)
clf_dt2tgt = GridSearchCV(dt, params, scoring = 'f1_micro', cv=2)

clf_dt1tgt.fit(X_train_tgt, y_train)
clf_dt2tgt.fit(X_train_tgt, y_train)

GridSearchCV(cv=2, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_samples_split': [2, 3, 4]}, scoring='f1_micro')

### Evaluate DecisionTree Feature 2

In [47]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_dt1tgt.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_dt2tgt.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

preds1 = clf_dt1tgt.predict(X_test_tgt)
preds2 = clf_dt2tgt.predict(X_test_tgt)

precision1 = precision_score(preds1, y_test, average = "macro") 
recall1 = recall_score(preds1, y_test, average = "macro")
f1_1 = f1_score(preds1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(preds2, y_test, average = "micro") 
recall2 = recall_score(preds2, y_test, average = "micro")
f1_2 = f1_score(preds2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4688
Validation F1_micro: 0.6875

F1 Macro
Precision: 0.2424
Recall: 0.3333
F1: 0.2807

F1 Micro
Precision: 0.7273
Recall: 0.7273
F1: 0.7273


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## DecisionTree with feature 3

In [55]:
from sklearn.tree import DecisionTreeClassifier
np.random.seed(42)
random.seed(42)

X_train_bdwdd = vec.fit_transform(X_txt_train)
X_test_bdwdd = vec.transform(X_txt_test)

arrTrain = np.array(X_train_bdwdd_features)
arrTest = np.array(X_test_bdwdd_features)

X_train_bdwd = hstack([X_train_bdwdd, arrTrain])
X_test_bdwd = hstack([X_test_bdwdd, arrTest])

params = {'min_samples_split': [2, 3, 4]}

dt = DecisionTreeClassifier(random_state=42)

# I added two gridsearchcv and fits so we can get both micro and macro

clf_dt1bdwd = GridSearchCV(dt, params, scoring = 'f1_macro', cv=2)
clf_dt2bdwd = GridSearchCV(dt, params, scoring = 'f1_micro', cv=2)

clf_dt1bdwd.fit(X_train_bdwd, y_train)
clf_dt2bdwd.fit(X_train_bdwd, y_train)

GridSearchCV(cv=2, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_samples_split': [2, 3, 4]}, scoring='f1_micro')

### Evaluate DecisionTree Feature 3

In [56]:
# again I doubled everything so we can get micro and macro scores

validation_score1 = clf_dt1bdwd.best_score_ 
print("Validation F1_macro: {:.4f}".format(validation_score1))

validation_score2 = clf_dt2bdwd.best_score_ 
print("Validation F1_micro: {:.4f}".format(validation_score2))

preds1 = clf_dt1bdwd.predict(X_test_bdwd)
preds2 = clf_dt2bdwd.predict(X_test_bdwd)

precision1 = precision_score(preds1, y_test, average = "macro") 
recall1 = recall_score(preds1, y_test, average = "macro")
f1_1 = f1_score(preds1, y_test, average = "macro")

print("\nF1 Macro")
print("Precision: {:.4f}".format(precision1))
print("Recall: {:.4f}".format(recall1))
print("F1: {:.4f}".format(f1_1))


precision2 = precision_score(preds2, y_test, average = "micro") 
recall2 = recall_score(preds2, y_test, average = "micro")
f1_2 = f1_score(preds2, y_test, average = "micro")

print("\nF1 Micro")
print("Precision: {:.4f}".format(precision2))
print("Recall: {:.4f}".format(recall2))
print("F1: {:.4f}".format(f1_2))

Validation F1_macro: 0.4653
Validation F1_micro: 0.6820

F1 Macro
Precision: 0.2355
Recall: 0.3333
F1: 0.2760

F1 Micro
Precision: 0.7119
Recall: 0.7119
F1: 0.7119


C:\Users\astar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#y_train_pred = cross_val_predict(clf, X_train_exc, y_train, cv=3)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

print(cross_val_score(clf_rf1tgt, X_train_tgt, y_train, scoring="f1_macro", cv=3))
print(cross_val_score(clf_rf1tgt, X_train_tgt, y_train, scoring="f1_micro", cv=3))

#train, validation = train_test_split(data, test_size=0.50, random_state = 5)

## Recode Test.txt Classes

In [ ]:
import csv

with open('test.tsv',"r", encoding="UTF-8") as source:
    reader= csv.reader(source, delimiter='\t')
    with open('test_pred.tsv', 'w', encoding="UTF-8") as result:
        writer=csv.writer(result, delimiter='\t')
        for row,i in zip(reader,preds2):
            row[2]=i
            #print(row)
            writer.writerows([row])